In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "sk-xxxx"
os.environ["SERPAPI_API_KEY"] = 'xxxx'

from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.llms import OpenAI
from langchain.agents import AgentType

# 加载 OpenAI 模型
llm = OpenAI(temperature=0,max_tokens=2048) 

 # 加载 serpapi 工具
tools = load_tools(["serpapi"])

# 如果搜索完想再计算一下可以这么写
# tools = load_tools(['serpapi', 'llm-math'], llm=llm)

# 如果搜索完想再让他再用python的print做点简单的计算，可以这样写
# tools=load_tools(["serpapi","python_repl"])

# 工具加载后都需要初始化，verbose 参数为 True，会打印全部的执行详情
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# 运行 agent
agent.run("What's the date today? ")

In [ ]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import DirectoryLoader
from langchain.vectorstores import Chroma, Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain

import pinecone
import os

os.environ["OPENAI_API_KEY"] = "sk-xxxx"
os.environ["SERPAPI_API_KEY"] = 'xxxxx'

# 初始化 pinecone
pinecone.init(
  api_key="xxxxxx",
  environment="us-west1-gcp-free"
)

loader = DirectoryLoader('/Users/puchunwei/sanbox/BigModelResearch/knowledge-repository', glob='*.txt')
# 将数据转成 document 对象，每个文件会作为一个 document
documents = loader.load()

# 初始化加载器
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=0)
# 切割加载的 document
split_docs = text_splitter.split_documents(documents)

print(f'documents:{len(split_docs)}')

embeddings = OpenAIEmbeddings()
index_name="vector-openai"

# 持久化数据
# docsearch = Pinecone.from_texts([t.page_content for t in split_docs], embeddings, index_name="vector-openai")

# 加载数据
docsearch = Pinecone.from_existing_index(index_name, embeddings)

query = "蒲春伟是谁？"
docs = docsearch.similarity_search(query)


llm = OpenAI(temperature=0)
chain = load_qa_chain(llm, chain_type="stuff", verbose=True)
chain.run(input_documents=docs, question=query)
